In [1]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;

/Users/danielwu/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from preprocess import preprocessData
[Dtrainmat_Diagnosis, Y_FutureDiagnosis, RID, Dtrainmat, Dtrain] = preprocessData()



Load data and select features
Train SVM for Diagnosis and SVR for ADAS and Ventricles


Using TensorFlow backend.


[MICE] Completing matrix with shape (12012, 22)
[MICE] Starting imputation round 1/110, elapsed time 0.005
[MICE] Starting imputation round 2/110, elapsed time 0.157
[MICE] Starting imputation round 3/110, elapsed time 0.229
[MICE] Starting imputation round 4/110, elapsed time 0.297
[MICE] Starting imputation round 5/110, elapsed time 0.359
[MICE] Starting imputation round 6/110, elapsed time 0.423
[MICE] Starting imputation round 7/110, elapsed time 0.488
[MICE] Starting imputation round 8/110, elapsed time 0.549
[MICE] Starting imputation round 9/110, elapsed time 0.610
[MICE] Starting imputation round 10/110, elapsed time 0.672
[MICE] Starting imputation round 11/110, elapsed time 0.736
[MICE] Starting imputation round 12/110, elapsed time 0.800
[MICE] Starting imputation round 13/110, elapsed time 0.863
[MICE] Starting imputation round 14/110, elapsed time 0.925
[MICE] Starting imputation round 15/110, elapsed time 0.991
[MICE] Starting imputation round 16/110, elapsed time 1.053
[

In [15]:
#Train SVM for diagnosis
import sklearn.svm as svm
clf = svm.SVC(kernel='linear',probability=True)
clf.fit(Dtrainmat_Diagnosis,Y_FutureDiagnosis)


model = RandomForestClassifier(n_estimators=200, criterion='entropy')
model.fit(Dtrainmat_Diagnosis,Y_FutureDiagnosis)


gradmodel = GradientBoostingClassifier()
gradmodel.fit(Dtrainmat_Diagnosis,Y_FutureDiagnosis)

GNB = GaussianNB()
gradmodel.fit(Dtrainmat_Diagnosis,Y_FutureDiagnosis)

LR = LogisticRegression()
LR.fit(Dtrainmat_Diagnosis,Y_FutureDiagnosis)

NameError: name 'GradientBoostingClassifier' is not defined

In [60]:
import os

def evaluate_data(Dtrainmat, RID, model):
    str_exp = '/Users/danielwu/Dropbox/Documents/CSCI5525/Project/'
    S = pd.read_csv(os.path.join(str_exp, 'IntermediateData',
                                 'ToPredict.csv'), header=None)
    S = S.values

    Dtestmat = np.zeros((len(S), Dtrainmat.shape[1]))
    # y_Dtestmat = np.zeros((len(S),1))
    for i in range(len(S)):
        idx_S = RID.values == S[i]
        Dtestmat[i, :] = Dtrainmat[np.where(idx_S)[0][-1], :]

    print("Running predictions on test set samples...")
    p=model.predict(Dtestmat)
    # print(results)
    # Write ouput format to files
    o = np.column_stack((S, S, S, p))
    count = 0
    years = [str(a) for a in range(2010, 2018)]
    months = [str(a).zfill(2) for a in range(1, 13)]
    ym = [y + '-' + mo for y in years for mo in months]
    ym = ym[4:-8]
    nr_pred = len(ym)
    o1 = np.zeros((o.shape[0] * nr_pred, o.shape[1]))
    ym1 = [a for b in range(0, len(S)) for a in ym]
    for i in range(len(o)):
        o1[count:count + nr_pred] = o[i]
        o1[count:count + nr_pred, 1] = range(1, nr_pred + 1)
        count = count + nr_pred

    output = pd.DataFrame(o1, columns=['RID', 'Forecast Month', 'Forecast Date','CN relative probability', 'MCI relative probability', 'AD relative probability'])
    output['Forecast Month'] = output['Forecast Month'].astype(int)
    output['Forecast Date'] = ym1

    str_out_final = os.path.join(str_exp, 'IntermediateData', 'TADPOLE_Submission_Leaderboard_NeuralNet.csv')
    output.to_csv(str_out_final, header=True, index=False)

    print('Evaluate predictions')
    R = pd.read_csv('./TADPOLE_LB4.csv')
    import evalOneSubmission as eos
    mAUC, bca = eos.evalOneSub(R, output)
    print('Diagnosis:')
    print('mAUC = ' + "%0.3f" % mAUC)
    print('BAC = ' + "%0.3f" % bca)
    return Dtestmat, p



In [21]:
# Some useful parameters which will come in handy later on
# ntrain = train.shape[0]
# ntest = test.shape[0]
SEED = 0 # for reproducibility
# NFOLDS = 5 # set folds for out-of-fold prediction
# kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict_proba(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

In [58]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 1000,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':1000,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 1000,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 1000,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025,
    'probability' : True,
    }

In [61]:
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

rf.train(Dtrainmat_Diagnosis,Y_FutureDiagnosis)
et.train(Dtrainmat_Diagnosis,Y_FutureDiagnosis)
ada.train(Dtrainmat_Diagnosis,Y_FutureDiagnosis)
gb.train(Dtrainmat_Diagnosis,Y_FutureDiagnosis)
svc.train(Dtrainmat_Diagnosis,Y_FutureDiagnosis)



In [7]:
rf_feature = rf.feature_importances(Dtrainmat_Diagnosis,Y_FutureDiagnosis)
et_feature = et.feature_importances(Dtrainmat_Diagnosis,Y_FutureDiagnosis)
ada_feature = ada.feature_importances(Dtrainmat_Diagnosis,Y_FutureDiagnosis)
gb_feature = gb.feature_importances(Dtrainmat_Diagnosis,Y_FutureDiagnosis)

[ 0.15203366  0.01036644  0.00664635  0.00509196  0.01170777  0.36911283
  0.00852614  0.01009624  0.11953384  0.09848231  0.0269923   0.0053619
  0.00157145  0.00533245  0.00522342  0.00487431  0.01296045  0.03671385
  0.02301571  0.02771496  0.03625818  0.02238347]
[ 0.25238362  0.02110649  0.00862483  0.00881248  0.01543271  0.03653952
  0.01152528  0.01227112  0.02839765  0.01655131  0.32053285  0.01326441
  0.0372636   0.01015432  0.00862629  0.00795869  0.03968165  0.0401349
  0.01645949  0.03964053  0.03317329  0.02046498]
[ 0.04   0.069  0.053  0.031  0.044  0.115  0.045  0.03   0.037  0.086
  0.013  0.067  0.002  0.044  0.035  0.057  0.03   0.034  0.039  0.041
  0.047  0.041]
[ 0.04992825  0.05942368  0.05177433  0.04319129  0.05397149  0.06708849
  0.04074799  0.04232416  0.03734032  0.05072857  0.00656549  0.0450618
  0.00344616  0.0484558   0.04671199  0.04199284  0.0514403   0.05492371
  0.05089077  0.05249779  0.05798786  0.04350692]


In [12]:
rf_features = [ 0.15203366,  0.01036644,  0.00664635,  0.00509196,  0.01170777,  0.36911283,
               0.00852614,  0.01009624,  0.11953384,  0.09848231,  0.0269923,   0.0053619,
               0.00157145,  0.00533245,  0.00522342,  0.00487431,  0.01296045,  0.03671385,
               0.02301571,  0.02771496,  0.03625818,  0.02238347]
et_features = [ 0.25238362,  0.02110649,  0.00862483,  0.00881248,  0.01543271,  0.03653952,
               0.01152528,  0.01227112,  0.02839765,  0.01655131 , 0.32053285 , 0.01326441,
               0.0372636,   0.01015432,  0.00862629,  0.00795869,  0.03968165,  0.0401349,
               0.01645949,  0.03964053,  0.03317329,  0.02046498]
ada_features = [ 0.04,   0.069,  0.053 , 0.031,  0.044,  0.115,  0.045,  0.03,   0.037 , 0.086,
                0.013,  0.067 , 0.002 , 0.044 , 0.035 , 0.057 , 0.03  , 0.034 , 0.039 , 0.041,
                0.047 , 0.041]
gb_features = [ 0.04992825,  0.05942368,  0.05177433 , 0.04319129 , 0.05397149 , 0.06708849,
               0.04074799,  0.04232416 , 0.03734032 , 0.05072857,  0.00656549,  0.0450618,
               0.00344616 , 0.0484558,   0.04671199,  0.04199284 , 0.0514403 ,  0.05492371,
               0.05089077,  0.05249779 , 0.05798786 , 0.04350692]

cols = Dtrain.columns.values
# Create a dataframe with features
feature_dataframe = pd.DataFrame( {'features': cols,
     'Random Forest feature importances': rf_features,
     'Extra Trees  feature importances': et_features,
      'AdaBoost feature importances': ada_features,
    'Gradient Boost feature importances': gb_features
    })

In [13]:
# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Random Forest feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Random Forest feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Extra Trees  feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Extra Trees  feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Extra Trees Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['AdaBoost feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['AdaBoost feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'AdaBoost Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Gradient Boost feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Gradient Boost feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Gradient Boosting Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [15]:
# Create the new column containing the average of values

feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise
feature_dataframe.head(3)

y = feature_dataframe['mean'].values
x = feature_dataframe['features'].values
data = [go.Bar(
            x= x,
             y= y,
            width = 0.5,
            marker=dict(
               color = feature_dataframe['mean'].values,
            colorscale='Portland',
            showscale=True,
            reversescale = False
            ),
            opacity=0.6
        )]

layout= go.Layout(
    autosize= True,
    title= 'Barplots of Mean Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='bar-direct-labels')

In [53]:
[Dtestmat, prf] = evaluate_data(Dtrainmat, RID, rf)
[Dtestmat, pet] = evaluate_data(Dtrainmat, RID, et)
[Dtestmat, pada] = evaluate_data(Dtrainmat, RID, ada)
[Dtestmat, pgb] = evaluate_data(Dtrainmat, RID, gb)
[Dtestmat, psvm] = evaluate_data(Dtrainmat, RID, svc)



Running predictions on test set samples...
Evaluate predictions
Diagnosis:
mAUC = 0.693
BAC = 0.535
Running predictions on test set samples...
Evaluate predictions
Diagnosis:
mAUC = 0.649
BAC = 0.543
Running predictions on test set samples...
Evaluate predictions
Diagnosis:
mAUC = 0.576
BAC = 0.540
Running predictions on test set samples...
Evaluate predictions
Diagnosis:
mAUC = 0.628
BAC = 0.571
Running predictions on test set samples...


AttributeError: predict_proba is not available when  probability=False

In [31]:

base_predictions_train = pd.DataFrame( {'RandomForest': rf.clf.predict(Dtrainmat_Diagnosis).ravel(),
     'ExtraTrees': et.clf.predict(Dtrainmat_Diagnosis).ravel(),
     'AdaBoost': ada.clf.predict(Dtrainmat_Diagnosis).ravel(),
      'GradientBoost': gb.clf.predict(Dtrainmat_Diagnosis).ravel()
    })
base_predictions_train.head()

,AdaBoost,ExtraTrees,GradientBoost,RandomForest
0,2.0,2.0,1.0,1.0
1,1.0,2.0,1.0,1.0
2,2.0,2.0,1.0,1.0
3,1.0,1.0,1.0,1.0
4,2.0,2.0,2.0,2.0


In [49]:
data = [
    go.Heatmap(
        z= base_predictions_train.astype(float).corr().values ,
        x=base_predictions_train.columns.values,
        y= base_predictions_train.columns.values,
          colorscale='Viridis',
            showscale=True,
            reversescale = True
    )
]
py.iplot(data, filename='labelled-heatmap')
x_test = np.concatenate(( rf.clf.predict(Dtestmat).reshape(-1,1), et.clf.predict(Dtestmat).reshape(-1,1), ada.clf.predict(Dtestmat).reshape(-1,1), gb.clf.predict(Dtestmat).reshape(-1,1), svc.clf.predict(Dtestmat).reshape(-1,1)), axis=1)

x_train = np.concatenate(( rf.clf.predict(Dtrainmat_Diagnosis).reshape(-1,1), et.clf.predict(Dtrainmat_Diagnosis).reshape(-1,1), ada.clf.predict(Dtrainmat_Diagnosis).reshape(-1,1), gb.clf.predict(Dtrainmat_Diagnosis).reshape(-1,1), svc.clf.predict(Dtrainmat_Diagnosis).reshape(-1,1)), axis=1)



In [52]:
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softmax',
 nthread= -1,
 scale_pos_weight=1).fit(x_train, Y_FutureDiagnosis)

str_exp = '/Users/danielwu/Dropbox/Documents/CSCI5525/Project/'
S = pd.read_csv(os.path.join(str_exp, 'IntermediateData',
                             'ToPredict.csv'), header=None)
S = S.values

Dtestmat = np.zeros((len(S), Dtrainmat.shape[1]))
# y_Dtestmat = np.zeros((len(S),1))
for i in range(len(S)):
    idx_S = RID.values == S[i]
    Dtestmat[i, :] = Dtrainmat[np.where(idx_S)[0][-1], :]

print("Running predictions on test set samples...")
p=gbm.predict_proba(x_test)
# print(results)
# Write ouput format to files
o = np.column_stack((S, S, S, p))
count = 0
years = [str(a) for a in range(2010, 2018)]
months = [str(a).zfill(2) for a in range(1, 13)]
ym = [y + '-' + mo for y in years for mo in months]
ym = ym[4:-8]
nr_pred = len(ym)
o1 = np.zeros((o.shape[0] * nr_pred, o.shape[1]))
ym1 = [a for b in range(0, len(S)) for a in ym]
for i in range(len(o)):
    o1[count:count + nr_pred] = o[i]
    o1[count:count + nr_pred, 1] = range(1, nr_pred + 1)
    count = count + nr_pred

output = pd.DataFrame(o1, columns=['RID', 'Forecast Month', 'Forecast Date','CN relative probability', 'MCI relative probability', 'AD relative probability'])
output['Forecast Month'] = output['Forecast Month'].astype(int)
output['Forecast Date'] = ym1

str_out_final = os.path.join(str_exp, 'IntermediateData', 'TADPOLE_Submission_Leaderboard_NeuralNet.csv')
output.to_csv(str_out_final, header=True, index=False)

print('Evaluate predictions')
R = pd.read_csv('./TADPOLE_LB4.csv')
import evalOneSubmission as eos
mAUC, bca = eos.evalOneSub(R, output)
print('Diagnosis:')
print('mAUC = ' + "%0.3f" % mAUC)
print('BAC = ' + "%0.3f" % bca)

Running predictions on test set samples...
Evaluate predictions
Diagnosis:
mAUC = 0.593
BAC = 0.571
